# Recurrent Neural Network

In [6]:
import numpy as np

In [15]:
# data I/O
data = open('input.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print 'data has %d characters, %d unique.' % (data_size, vocab_size)
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

data has 1726 characters, 45 unique.


In [16]:
print char_to_ix

{'\x80': 0, '\n': 1, '\x99': 2, ' ': 3, ')': 4, '(': 5, ',': 6, '.': 7, '1': 8, '5': 9, 'D': 10, 'F': 11, 'I': 12, 'M': 13, 'L': 14, 'S': 15, '\xe2': 22, 'T': 17, 'W': 18, 'Y': 19, 'a': 20, 'c': 21, 'b': 16, 'e': 23, 'd': 24, 'g': 25, 'f': 26, 'i': 27, 'h': 28, 'k': 29, 'm': 30, 'l': 31, 'o': 32, 'n': 33, 'q': 34, 'p': 35, 's': 36, 'r': 37, 'u': 38, 't': 39, 'w': 40, 'v': 41, 'y': 42, 'x': 43, 'z': 44}


In [17]:
print ix_to_char

{0: '\x80', 1: '\n', 2: '\x99', 3: ' ', 4: ')', 5: '(', 6: ',', 7: '.', 8: '1', 9: '5', 10: 'D', 11: 'F', 12: 'I', 13: 'M', 14: 'L', 15: 'S', 16: 'b', 17: 'T', 18: 'W', 19: 'Y', 20: 'a', 21: 'c', 22: '\xe2', 23: 'e', 24: 'd', 25: 'g', 26: 'f', 27: 'i', 28: 'h', 29: 'k', 30: 'm', 31: 'l', 32: 'o', 33: 'n', 34: 'q', 35: 'p', 36: 's', 37: 'r', 38: 'u', 39: 't', 40: 'w', 41: 'v', 42: 'y', 43: 'x', 44: 'z'}


In [18]:
# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 20 # number of steps to unroll the RNN for
learning_rate = 1e-1

In [19]:

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias


In [20]:

def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in xrange(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(xrange(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [21]:
def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in xrange(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

In [22]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while True:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
  if n % 100 == 0:
    sample_ix = sample(hprev, inputs[0], 100)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print '----\n %s \n----' % (txt, )

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 100 == 0: print 'iter %d, loss: %f' % (n, smooth_loss) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 

----
 FeopIwmu1rY51M,v.q.vw5eti aDMD�m .YWSw)� vIFv r,eti
Mr�xS Lvatlgatyc�aS5cazM,
f�
��,Moki.FpvYI�We S� 
----
iter 0, loss: 76.133252
----
 pmleamterfSees epsapleieegeheeehsshhennshlppheppteabgphgheeeeeetphrteirhirpalrpsehpepetebsembhephes  
----
iter 100, loss: 76.716273
----
 inifn  nrawrt.nleTraorofniunitroel,.a  nt cr corrhanl c impaiooeastgceauhqfrolre gnceenl,ie t,georpa 
----
iter 200, loss: 75.649193
----
 nde caoro  eofer s et t
se e heo)ole le taemtfaieg rop  sri m peeief ui   o m tpyoelee poeeiaoell nd 
----
iter 300, loss: 74.303432
----
 we tuaoshet ooer he t Lftse 
weig ry sTosceTo  rewhe thMLweur siayboSeoYei Sy murosrehe yogcolhens r 
----
iter 400, loss: 72.928923
----
  �h�od eureaohe cheosot te nngatcerhash t rhf neceet thenrtetbyre v �xadhe ts e Sleed, nive ada it l 
----
iter 500, loss: 71.436425
----
 ngonanme onFinorna g nnlzaunhnnl vct nsnnTnnhy(n ztgsuan n z�ld bialhnurvipinngenfes tudan alithnnnc 
----
iter 600, loss: 69.950229
----
 eacatoTke ip, bes. fe p

KeyboardInterrupt: 